# Experimentação - Testes A/B




bonferroni modelagem

ana, renato  maria, franco, nuno


In [1]:
import scipy.stats as st

def sample_size_binomial(alpha, ME, p):
    import scipy.stats as st
    import numpy as np
    z = np.abs(st.norm.ppf((alpha)/2))
    nc = np.ceil(((z/ME)**2)*2*p*(1 - p))
    return(nc)

In [ ]:
sample_size_binomial(0.05, 0.02, p = 0.1)

## Experimento 1

Uma plataforma de vídeos cuja principal fonte de receita é propaganda. Suponha que ganhamos R$1.5 a cada segundo de ad visto por um cliente. 

A plataforma quer saber se deve aumentar a quantidade de ads. 

In [ ]:
import datetime
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# créditos ao Hector Lira

np.random.seed(200)
random.seed(200)

n_customers = 1000

customer_ids = []
experiment_groups = []
timestamps = []
videos_seen_logs = []
ads_time_logs = []
ages = []
genders = []

start_date = datetime.datetime(2021, 6, 1, 00, 00)

for i in range(n_customers):

  experiment_group = random.choices(["control", "variant"])[0]
  age = int(np.random.uniform(low=18, high=60))
  gender = random.choices(["m", "w"], weights = [0.5, 0.5])[0]

  n_events = int(np.random.uniform(low=1, high=30))
  timestamp_event = start_date

  for j in range(n_events):
    random_range = np.random.uniform(low=1, high=10)
    timestamp_event = timestamp_event + datetime.timedelta(days=random_range)

    if experiment_group == "control":
      if age <= 35:
        ads_time = np.random.uniform(low=5, high=10)
        videos_seen = np.random.uniform(low=3, high=7)
      elif 35 < age < 45:
        ads_time = np.random.uniform(low=5, high=10)
        videos_seen = np.random.uniform(low=2, high=6)
      else:
        ads_time = np.random.uniform(low=5, high=10)
        videos_seen = np.random.uniform(low=1, high=4)

    else:
      if age <= 35:
        ads_time = np.random.uniform(low=10, high=15)
        videos_seen = np.random.uniform(low=1, high=5)
      elif 35 < age <= 45:
        ads_time = np.random.uniform(low=10, high=15)
        videos_seen = np.random.uniform(low=2, high=5)
      else:
        ads_time = np.random.uniform(low=10, high=15)
        videos_seen = np.random.uniform(low=1, high=4)

    customer_ids.append(i + 1)
    experiment_groups.append(experiment_group)
    timestamps.append(timestamp_event.strftime("%d/%m/%y %H:%M"))
    videos_seen_logs.append(round(videos_seen))
    ads_time_logs.append(round(ads_time, 2))
    ages.append(age)
    genders.append(gender)


df_videos = pd.DataFrame(data={
  "customer_id": customer_ids,
  "experiment_group": experiment_groups,
  "timestamp": timestamps,
  "videos_seen": videos_seen_logs,
  "avg_ads_time": ads_time_logs,
  "gender": genders,
  "age": ages
})

In [ ]:
df_videos.head()

In [ ]:
gender = df_videos.groupby(['experiment_group', 'gender'])['customer_id'].count().reset_index()
gender['customer_id'] =  gender['customer_id'] / gender.groupby(['experiment_group'])['customer_id'].transform('sum')
gender.sort_values('gender', inplace = True)

In [ ]:
gender

In [ ]:
from scipy.stats import chisquare
chisquare(gender.query('experiment_group == "variant"')['customer_id'], gender.query('experiment_group == "control"')['customer_id'])

In [ ]:
df_videos['revenue'] = df_videos['avg_ads_time'] * df_videos['videos_seen'] * 1.5

In [ ]:
df_videos.groupby('experiment_group')[['videos_seen', 'avg_ads_time', 'revenue']].mean()

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
ttest_ind(df_videos.query('experiment_group == "variant"')['revenue'], df_videos.query('experiment_group == "control"')['revenue'])

## Experimento 2

E-commerce. Queremos saber se uma alteração no site da empresa resultará em maiores vendas


In [ ]:
#@title

##### NÃO MODIFICAR ESTE CÓDIGO

# créditos ao Hector Lira

np.random.seed(100)
random.seed(100)
experiment_launch = "2021-06-01"
n_customers = 10000

customer_ids = []
visit_ids = []
experiment_groups = []
dates = []
all_purchases = []
total_sales = []
ages = []
genders = []

start_date = datetime.datetime(2021, 1, 1, 00, 00)


def get_sale_simulation(age, control=True):

  purchase_modifier = (not control) * 1

  if age <= 35:
    purchases = round(np.random.uniform(low=1, high=5 + purchase_modifier), 0)
    avg_ticket = round(np.random.uniform(low=25, high=50), 2)
  elif 35 < age < 45:
    purchases = round(np.random.uniform(low=1, high=10 + purchase_modifier), 0)
    avg_ticket = round(np.random.uniform(low=30, high=60), 2)
  else:
    purchases = round(np.random.uniform(low=2, high=15 + purchase_modifier), 0)
    avg_ticket = round(np.random.uniform(low=30, high=60), 2)

  total_sale = round(purchases * avg_ticket, 2)

  return purchases, avg_ticket, total_sale


for i in range(n_customers):

  experiment_group = random.choices(["control", "variant"])[0]
  age = int(np.random.uniform(low=18, high=60))
  gender = random.choices(["m", "w"], weights = [0.5, 0.5])[0]

  n_events = int(np.random.uniform(low=1, high=15))

  for j in range(n_events):
    random_range = np.random.uniform(low=1, high=360)
    date_event = start_date + datetime.timedelta(days=random_range)

    if (
        date_event.strftime("%Y-%m-%d") < experiment_launch or
        experiment_group == "control"
    ):
        purchases, avg_ticket, total_sale = get_sale_simulation(age)
    else:
        purchases, avg_ticket, total_sale = get_sale_simulation(age, False)

    customer_ids.append(i + 1)
    experiment_groups.append(experiment_group)
    dates.append(date_event.strftime("%Y-%m-%d"))
    all_purchases.append(purchases)
    total_sales.append(total_sale)
    ages.append(age)
    genders.append(gender)


df_purchases = pd.DataFrame(data={
  "customer_id": customer_ids,
  "experiment_group": experiment_groups,
  "date": dates,
  "purchases": all_purchases,
  "total_sale": total_sales,
  "age": ages,
  "gender": genders,
}).sort_values(by=["customer_id", "date"])

df_purchases["date"] = pd.to_datetime(df_purchases["date"])

df_purchases["week"] = df_purchases["date"] - df_purchases["date"].dt.weekday * np.timedelta64(1, 'D')

In [ ]:
df_purchases.head()

In [ ]:
df_purchases[["customer_id", "age", "experiment_group"]].drop_duplicates().groupby("experiment_group").agg({
    "age": "mean",
    "customer_id": "count"
}).reset_index()

In [ ]:
pre_experiment_data = df_purchases[df_purchases["date"] < experiment_launch]

pre_experiment_data.groupby("experiment_group").agg({
    "purchases": "mean",
    "total_sale": "mean"
}).reset_index()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

sns.barplot(data=pre_experiment_data, y="total_sale", x="experiment_group", ax=ax)
ax.set_ylim(225, 250)

fig.tight_layout(pad=3.0)

In [ ]:
ttest_ind(pre_experiment_data.query('experiment_group == "variant"')['total_sale'], pre_experiment_data.query('experiment_group == "control"')['total_sale'])

In [ ]:
post_experiment_data = df_purchases[df_purchases["date"] >= experiment_launch]

post_experiment_data.groupby("experiment_group").agg({
    "purchases": "mean",
    "total_sale": "mean"
}).reset_index()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

sns.barplot(data=post_experiment_data, y="total_sale", x="experiment_group", ax=ax)
ax.set_ylim(220, 280)

fig.tight_layout(pad=3.0)

In [ ]:
ttest_ind(post_experiment_data.query('experiment_group == "variant"')['total_sale'], post_experiment_data.query('experiment_group == "control"')['total_sale'])

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

sns.lineplot(data=df_purchases, y="total_sale", x="week", hue="experiment_group",ax=ax)
ax.axvline(datetime.datetime(2021, 6, 1), color="red", ls="--")
sns.despine()
fig.tight_layout(pad=3.0)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

sns.lineplot(data=df_purchases, y="purchases", x="week", hue="experiment_group",ax=ax)
ax.axvline(datetime.datetime(2021, 6, 1), color="red", ls="--")
sns.despine()
fig.tight_layout(pad=3.0)

In [ ]:
weeks = []
pvalues_sales = []
pvalues_volume = []
for week, df_week in df_purchases.groupby('week'):
  weeks.append(week)
  pvalues_sales.append(ttest_ind(df_week.query('experiment_group == "variant"')['total_sale'], df_week.query('experiment_group == "control"')['total_sale'])[-1])
  pvalues_volume.append(ttest_ind(df_week.query('experiment_group == "variant"')['purchases'], df_week.query('experiment_group == "control"')['purchases'])[-1])

pvalue_df = pd.DataFrame({'week' : weeks, 'p-values_sales' : pvalues_sales, 'p-values_volume' : pvalues_volume})

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

sns.lineplot(data=pvalue_df, y="p-values_sales", x="week", ax=ax, label = 'sales')
sns.lineplot(data=pvalue_df, y="p-values_volume", x="week", ax=ax, label = 'volume')
ax.axvline(datetime.datetime(2021, 6, 1), color="red", ls="--")
ax.axhline(0.05, color="red", ls="--")
plt.legend()
sns.despine()
fig.tight_layout(pad=3.0)